In [ ]:
import onnx
import onnx.helper as helper

def modify_onnx_model(input_path, output_path):
    # Load ONNX model
    model = onnx.load(input_path)
    graph = model.graph
    
    # Find the logits output node
    logits_output = graph.output[0]  # Assuming first output is logits
    logits_name = logits_output.name

    # Define Softmax node
    softmax_output_name = "softmax_output"
    softmax_node = helper.make_node(
        "Softmax",
        inputs=[logits_name],
        outputs=[softmax_output_name],
        axis=1  # Softmax along channel dimension (assuming NCHW format)
    )

    # Define ArgMax node
    mask_output_name = "segmentation_mask"
    argmax_node = helper.make_node(
        "ArgMax",
        inputs=[softmax_output_name],
        outputs=[mask_output_name],
        axis=1,  # Get the most probable class per pixel
        keepdims=0  # Remove extra dimension
    )
    
    # Add new nodes to the graph
    graph.node.extend([softmax_node])
    
    # Modify the model outputs
    graph.output[0].name = softmax_output_name
    graph.output[0].type.tensor_type.elem_type = onnx.TensorProto.FLOAT # Set output to INT8

    # Save the modified model
    onnx.save(model, output_path)
    print(f"Modified model saved to {output_path}")

# Example usage
output_model= "plaiades-voirie-lc1.onnx"
# modify_onnx_model("/home/adelb/Downloads/v512_pleiades_voirie_200eps_wabloss_model3r_lc1_3c_se_resnext50_32x4d_512.onnx", output_model)


Modified model saved to plaiades-voirie-lc1.onnx


In [1]:
import json
import onnx

input_model = "spot.onnx"
model = onnx.load(input_model)

# with open('../VGG-PROCESSING/all_autres_classes.txt') as f:
#     classes= f.read().split()

classes= ['background', "Bati", "voirie"]

class_names= {i: cls for i, cls in enumerate(classes)}

m1 = model.metadata_props.add()
m1.key = 'model_type'
m1.value = json.dumps('Segmentor')

m2 = model.metadata_props.add()
m2.key = 'class_names'
m2.value = json.dumps(class_names)

m3 = model.metadata_props.add()
m3.key = 'resolution'
m3.value = json.dumps(150.0)

# optional, if you want to standarize input after normalisation
m4 = model.metadata_props.add()
m4.key = 'standardization_mean'
m4.value = json.dumps( [0.485, 0.456, 0.406])

m5 = model.metadata_props.add()
m5.key = 'standardization_std'
m5.value = json.dumps([0.229, 0.224, 0.225])

m6 = model.metadata_props.add()
m6.key = 'tiles_size'
m6.value = json.dumps(512)

m7 = model.metadata_props.add()
m7.key = 'tiles_overlap'
m7.value = json.dumps(0)

onnx.save(model, 'nspot.onnx')

In [12]:
import onnx
from onnx import helper

# Load the ONNX model
model_path = output_model
model = onnx.load(model_path)
graph = model.graph

# Identify the node to remove
node_to_remove = "/Div"  # Change this to the actual node name

# Find the node object
node_to_delete = None
for node in graph.node:
    if node.name == node_to_remove:
        node_to_delete = node
        break

if node_to_delete is None:
    raise ValueError(f"Node '{node_to_remove}' not found in the model.")

print(node_to_delete)

# Get the inputs and outputs of the node to be deleted
input_name = node_to_delete.input[0]  # Assuming a single input
output_name = node_to_delete.output[0]  # Assuming a single output

# Update nodes that use the deleted node's output as input
for node in graph.node:
    for i, input in enumerate(node.input):
        if input == output_name:
            node.input[i] = input_name  # Redirect input to bypass deleted node

# Remove the node from the graph
new_nodes = [node for node in graph.node if node.name != node_to_remove]
graph.ClearField("node")
graph.node.extend(new_nodes)

# Save the modified model
onnx.save(model, output_model)
print(f"Node '{node_to_remove}' removed and connections updated successfully.")


input: "input"
input: "norm"
output: "/Div_output_0"
name: "/Div"
op_type: "Div"

Node '/Div' removed and connections updated successfully.


In [4]:
l = [1,2,3,4,5,6,7,8,9]
l[2:5][0]

3